In [6]:
#import urllib2
from bs4 import BeautifulSoup
import pandas as pd
import requests
from lxml import html
import string

In [27]:
# Grab unique url segments for each restaurant that we want to scrape as a list
# CUISINES = ['woorijip-authentic-korean-food-new-york',
#                'barn-joo-35-new-york',
#                'her-name-is-han-new-york-3',
#                'jongro-bbq-new-york-3',
#                'tofu-tofu-new-york-5', 
#                'aska-brooklyn-3', 
#                'aquavit-new-york', 
#                'nordic-preserves-fish-and-wildlife-company-new-york', 
#                'agern-new-york', 
#                'the-copenhagen-new-york', 
#                'the-masalawala-new-york-2', 
#                'bk-jani-bushwick', 
#                'lahore-deli-new-york', 
#                'lahori-chilli-brooklyn-2', 
#                'kababish-jackson-heights', 
#                'ipanema-restaurant-new-york', 
#                'samba-kitchen-new-york-5', 
#                'villa-brazil-cafe-grill-astoria', 
#                'beco-brooklyn', 
#                'berimbau-do-brasil-new-york']

CUISINES = []

#create a base url that will iterate later with the unique url segments we have in our CUISINES list
base_urls = [ 'http://www.yelp.com/biz/%s' % s for s in CUISINES]
reviews_per_page = 300 # Number of reviews we want for each page
order_sequence = range(0, (reviews_per_page+1), 40)

In [28]:
review_list = []
date_list = []
cuisine_list = []
for ur in base_urls:
    for o in order_sequence:
        page = requests.get(ur + ("?start=%s" % o))
        tree = html.fromstring(page.text)
        
        # this will make a list of all the reviews
        reviews = tree.xpath('//p[@itemprop="description"]/text()')
        
        
        for rev in reviews:
            rev_listry = "".join(filter(lambda x: x in string.printable, rev))
            rev_listry = rev_listry.replace(",",":")
            rev_listry = rev_listry.replace("\'","")
            review_list.append(rev_listry) 
        
        date = tree.xpath('//meta[@itemprop="datePublished"]')
        
        
        for i in range(len(date)):
            date_list.append(date[i].get("content"))
        
        
        for i in range(len(date)):
            cuisine_list.append(ur)

In [29]:
reviews_df = pd.DataFrame()

reviews_df['review'] = review_list
reviews_df['date'] = date_list
reviews_df['restaurants'] = cuisine_list

In [30]:
reviews_df.head()

,review,date,restaurants
0,So Woorijip is definitely not one of those fan...,2019-11-20,http://www.yelp.com/biz/woorijip-authentic-kor...
1,They have a variety of Korean dishes to choose...,2019-11-10,http://www.yelp.com/biz/woorijip-authentic-kor...
2,When I went with my sister on a brief trip to ...,2019-11-02,http://www.yelp.com/biz/woorijip-authentic-kor...
3,Great place for fast comfort korean food! The ...,2019-11-26,http://www.yelp.com/biz/woorijip-authentic-kor...
4,I have definitely been to this shop many many ...,2019-10-23,http://www.yelp.com/biz/woorijip-authentic-kor...


In [31]:
reviews_df.to_csv('review123.csv')